In [1]:
#!pip install -q transformers faiss-cpu sentence-transformers
#!pip install -U bitsandbytes

!pip install -q streamlit ctransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [3]:
%%writefile app.py
# app.py
import streamlit as st
from ctransformers import AutoModelForCausalLM

# 1) Load model using ctransformers
@st.cache_resource
def load_model():
    return AutoModelForCausalLM.from_pretrained(
        "TheBloke/Llama-2-7B-Chat-GGML",
        model_file="llama-2-7b-chat.ggmlv3.q4_0.bin",  # You must download this model
        model_type="llama",
        gpu_layers=32  # Adjust for your GPU
    )

llm = load_model()

# 2) Streamlit UI
st.title("🧠 Mental Health Counselor Chatbot")
st.caption("💬 Powered by ctransformers LLaMA-2")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Display previous chat
for role, msg in st.session_state.chat_history:
    with st.chat_message(role):
        st.markdown(msg)

# New input
if user_input := st.chat_input("You:"):
    st.chat_message("user").markdown(user_input)
    st.session_state.chat_history.append(("user", user_input))

    # Rebuild prompt
    MAX_HISTORY_TURNS = 4
    hist = st.session_state.chat_history[-MAX_HISTORY_TURNS*2:]
    prompt = ""
    for role, msg in hist:
        prompt += f"{'User' if role=='user' else 'Bot'}: {msg}\n"
    prompt += "Bot:"

    # Generate reply
    reply = llm(prompt, max_new_tokens=150)
    reply = reply.split("User:")[0].strip()

    st.chat_message("assistant").markdown(reply)
    st.session_state.chat_history.append(("assistant", reply))



Writing app.py


In [4]:
!npm install localtunnel
!streamlit run app.py --server.address=localhost &>/content/logs.txt &
!npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 1s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋⠙35.194.239.179your url is: https://slick-donkeys-hammer.loca.lt


In [2]:
!pip show streamlit
!pip show ctransformers

Name: streamlit
Version: 1.45.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
Name: ctransformers
Version: 0.2.27
Summary: Python bindings for the Transformer models implemented in C/C++ using GGML library.
Home-page: https://github.com/marella/ctransformers
Author: Ravindra Marella
Author-email: mv.ravindra007@gmail.com
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, py-cpuinfo
Required-by: 
